In [ ]:
# Импортирование необходимых библиотек.
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
import numpy as np
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import BDeuScore, K2Score, BicScore
from pgmpy.estimators import HillClimbSearch
import seaborn as sns
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error,accuracy_score
from sklearn.preprocessing import KBinsDiscretizer
import time
import warnings

# Игнорирование возникающих предупреждений.
warnings.filterwarnings('ignore')

In [ ]:
# Загрузка датасета.
data = pd.read_csv('./data/vk_interests_finance.csv')
data.head()

In [ ]:
# Оставление необходимых столбцов.
data = data[['sex', 'relation', 'is_parent', 'has_pets', 'age', 'mean_tr', 'median_tr', 'tr_per_month']]

# Получение информации о датасете.
data.info()

In [ ]:
# Вывод оставшегося датасета.
data.head()

In [ ]:
# Дискретизация непрерывных значений датасета.
vk_data = data.copy()
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
data_discrete = est.fit_transform(vk_data.values[:,4:8])
vk_data[['age', 'mean_tr',	'median_tr', 'tr_per_month']] = data_discrete

In [ ]:
vk_data.head()

In [ ]:
# Функция для получения структуры байесовской сети.
def BNS(d: pd.DataFrame, score: str = 'bdeuscore'):
    hc = HillClimbSearch(d)
    model = hc.estimate(scoring_method=score)
    bn = BayesianModel(model.edges())

    return bn

In [ ]:
# Функция для рассчета среднего размера Марковского одеяла.
def AMBS(bn: BayesianModel):
    markov_blankets = []
    for node in bn.nodes():
        markov_blankets.append(len(bn.get_markov_blanket(node)))

    AMBS = sum(markov_blankets)/len(markov_blankets)

    return AMBS

In [ ]:
# Функция для рассчета силы связи.
def ES(d: pd.DataFrame, bn: BayesianModel):
    ES = BDeuScore(d).score(bn) / (d.shape[0] * len(bn.edges()))

    return ES

In [ ]:
# Функция для разделения датасета на слайсы.
def slice_me_nice(d: pd.DataFrame):
    # Параметры для рассчета размера слайсов:
    # лучший средний размер марковского одеяла.
    bestAMBS = 1
    
    # лучший показатель силы связи.
    bestES = -1

    # шаг.
    step = 0

    # Изначальный размер разбиения.
    sliceSize = 1 * vk_data.shape[1]

    # Первые sliceSize строк из датасета.
    d_sliced = d[:sliceSize].copy()

    print(len(d_sliced))

    # Получение структуры байесовской сети.
    BN = BNS(d_sliced)

    # Рассчет текущего среднего размера Марковского одеяла.   
    currentAMBS = AMBS(BN)

    # Рассчет текущего значения силы связи.
    currentES = ES(d_sliced, BN)

    # Количество шагов цикла.
    mstep = 100

    # ТОчности 

    # Для 8 слайсов 
    # # - для AMBS;
    e1 = 0.5

    # # - для ES.
    e2 = 0.05

    # Для 16 слайсов 
    # # - для AMBS;
   # e1 = 0.5

    # # - для ES.
    #e2 = 0.05

    # Запуск цикла для поиска лучших значений AMBS и ES.
    while ((step <= mstep) and 
    ((abs(currentAMBS-bestAMBS)>e1) or 
    (abs(currentES-bestES)>e2))):
        print(step)
        sliceSize = sliceSize * 2
        bestAMBS = currentAMBS
        bestES = currentES
        d_sliced = d[:sliceSize].copy()
        BN = BNS(d_sliced)
        currentAMBS = AMBS(BN)
        currentES = ES(d_sliced, BN)
        step = step + 1
        print(sliceSize, bestAMBS, bestES)
        print(abs(currentAMBS-bestAMBS), abs(currentES-bestES))
        print(d.shape[0] / (3 * d_sliced.shape[0]))

    # Получение достаточного размера датасета для обучения.
    ALS = d_sliced.shape[0]
    print(d_sliced.shape[0])

    # Вычисление числа слоев.
    num_d = d.shape[0] / (3 * ALS)

    return num_d

In [ ]:
num_d = slice_me_nice(vk_data)

In [ ]:
print('Число слоев: ', num_d)

In [ ]:
# Разделение датасета на слайсы.
size = int(len(vk_data) / num_d)-1
data_slices = [vk_data.loc[i:i+size-1,:] for i in range(0, len(vk_data),size)]

In [ ]:
print("Размер vk_data: ", len(data_slices[0]))

In [ ]:
print("Размер последнего слайса: ", len(data_slices[int(num_d)]))

In [ ]:
# Функция для перевода матрицы смежности в байесовскую сеть.
def AM_to_BN(am: np.array):
    edges = []
    nodes = ['sex', 'relation', 'is_parent', 'has_pets', 'age', 'mean_tr', 'median_tr', 'tr_per_month']

    for i in range(len(am)):
        for j in range(len(am)):
            if am[i][j] == 1:
                try:
                    edges.append((nodes[i], nodes[j]))
                    bn = BayesianModel(edges)

                except ValueError:
                    print('Beware the loop!')
    
    bn = BayesianModel(edges)

    return bn

In [ ]:
# Функция для возвращения матрицы смежности.
def AM(bns: BayesianModel):    
    try:
        nodes = ['sex', 'relation', 'is_parent', 'has_pets', 'age', 'mean_tr', 'median_tr', 'tr_per_month']
        matrix = np.zeros((8, 8))
        for i in range(len(nodes)):
            for j in range(len(nodes)):
                if nodes[j] in bns.get_children(nodes[i]):
                    matrix[i][j] = 1
    
    except AttributeError:
        matrix = np.zeros((8, 8))
        print(type(bns), bns)
    
    # Для теста.
    # print("\n", matrix, "\n")

    return matrix

In [ ]:
# Функция сложения нескольких матриц смежностей в одну.
def structure_ensemble(bns, data_slice, t):
    am = []
    es = []
    w = []
    wam = []

    # Для теста.
    # print("\n Количество сетей, поданных на сложение: ", len(bns), "\n")

    for bn in bns:
        am.append(AM(bn))
        es.append(ES(data_slice, bn))

    for i in range(len(bns)):
        w.append(es[i]/sum(es))
        wam.append(am[i] * w[i])

    fwam = sum(wam)
    y = t * min(w)
    bn = fwam.copy()

    for i in range(len(fwam)):
        for j in range(len(fwam)):
            if fwam[i,j] > y:
                try:
                    bn[i,j] = 1
                    AM_to_BN(bn)
                except ValueError:
                    bn[i,j] = 0
                    
            else:
                bn[i,j] = 0

    return bn

In [ ]:
# Функция для 
def data_slice_learner(data_slice: pd.DataFrame):
    bn_hc_bdeu = BNS(data_slice)
    print("\n BDeu \n", bn_hc_bdeu.edges,  "\n")
    bn_hc_k2 = BNS(data_slice, score='k2score')
    print("\n k2 \n", bn_hc_k2.edges,  "\n")
    bn_hc_bic = BNS(data_slice, score='bicscore')
    print("\n bic \n", bn_hc_bic.edges,  "\n")
    print("\n Размер слайса: ", len(data_slice))

    bns = [bn_hc_bdeu, bn_hc_k2, bn_hc_bic]
    t = 2
    bn = structure_ensemble(bns, data_slice, t)

    return bn

In [ ]:
# Функция выполгяющая структурное обучение в роли локальноого учителя.
def local_learner(data_slices: pd.DataFrame):
    es = []
 
    for data_slice in data_slices:
        bn_ds = data_slice_learner(data_slice)

        print("Матрица смежности локального слоя: \n", bn_ds, "\n")

        bn = AM_to_BN(bn_ds)
        es.append(ES(data_slice, bn))

    best_ds = max(es)

    return [bn_ds, best_ds]

In [ ]:
# Для сложения локальных сетей в глобальную.
def global_ensemble(local_structures, best_data_sclice, k=3):
    bn_final = structure_ensemble(local_structures, best_data_sclice, k)

    print("\nФинальная матрица смежности: \n", bn_final, "\n")

    raw_labels = ['sex', 'relation', 'is_parent', 'has_pets', 'age', 'mean_tr', 'median_tr', 'tr_per_month']

    bn_final = AM_to_BN(bn_final)
    G_K = nx.DiGraph()
    G_K.add_edges_from(bn_final.edges())
    pos = nx.layout.circular_layout(G_K)
    nx.draw(G_K, pos, node_size=10, font_size=10, with_labels=True, font_weight='bold')

    return bn_final

In [ ]:
# Разделение на локальных учителей.
local_structures = []
es = []
i = 0
tic = time.time()
for data_slice in data_slices:
    if i<int(num_d):
        info = local_learner([data_slice.loc[:,:]])
        local_structures.append(AM_to_BN(info[0]))
        es.append(info[1])
    
    i = i + 1

print("\n\n Время выполнения: ", time.time()-tic)

In [ ]:
# Слайсы, общий сбор!
bn = global_ensemble(local_structures, data_slices[es.index(max(es))], 2*3/3)

In [ ]:
# Разделение выборки на тренировочную и тестовую.
data_train, data_test = sklearn.model_selection.train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
data_discrete = est.fit_transform(data_train.values[:,4:8])
data_train[['age', 'mean_tr',	'median_tr', 'tr_per_month']] = data_discrete

In [ ]:
# Обучение сети.
bn.fit(data_train)

In [ ]:
testformed_data = data_test.copy()
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
data_discrete = est.fit_transform(testformed_data.values[:,4:8])
testformed_data[['age', 'mean_tr',	'median_tr', 'tr_per_month']] = data_discrete

In [ ]:
i = 0
times = []

# Запуск цикла для удаления и восстановления каждого параметра.
for column in testformed_data.columns:
    mis_data = testformed_data.drop(columns=[column])

    print('\n Потерян: ', column, '\n')

    # Заполнение пропусков в сетях.
    tic = time.time()
    result_data = bn.predict(mis_data)
    print('\n\n Время выполнения: ', time.time()-tic)
    times.append(time.time()-tic)

    result_data.reset_index(drop=True, inplace=True)
    testformed_data.reset_index(drop=True, inplace=True)

    # Проверка типа утеряного параметра.
    if vk_data[column].dtypes == 'object':
        regime = le[i].inverse_transform(result_data.values.astype(int))
            
        # Сравнение категориальных признаков.
        print('\n Accuracy: ', accuracy_score(x_test[column], regime), '\n')

    else:
        parameters = ['age', 'mean_tr',	'median_tr', 'tr_per_month']
        if column in parameters:
            concat_data = testformed_data[parameters]
            concat_data[column] = result_data[column]

            porosity = est.inverse_transform(concat_data)
            check_data = pd.DataFrame(data=porosity, columns=parameters)
            
        else:
            check_data = result_data

        # Подсчет среднеквадратической ошибки.
        print('\n Среднеквадратическая ошибка: ', mean_squared_error(result_data[column], data_test[column], squared=False), '\n')
    
    if vk_data[column].dtypes == 'object':
        i += 1

In [ ]:
# Среднее время предсказания.
print("\n Среднее время предсказания: ", np.mean(times), "\n")

In [ ]:
hc = HillClimbSearch(data_train)

In [ ]:
# Создание списка оценок для hill-climbing.
scores = ['k2score', 'bdeuscore', 'bicscore']

# Создание списка методов структурного обучения.
methods = ['HC + K2', 'HC + BDeu', 'HC + BIC']

# Создание списка моделей.
models = []

In [ ]:
# Создание структур.
for score in scores:
    tic = time.time()
    models.append(hc.estimate(scoring_method=score))
    print('\n\n HC + Оценка: ',  score, '. Время выполнения: ', time.time()-tic)

In [ ]:
# Создание списка байесовских сетей.
bns = []

for i in range(len(models)):
    bns.append(BayesianModel(models[i].edges()))

In [ ]:
# Обучение байесовских сетей.
for i in range(len(bns)):
    tic = time.time()
    bns[i].fit(data_test)
    print('\n\n Hill-climb + Оценка: ',  scores[i], '. Время выполнения: ', time.time()-tic)

In [ ]:
# Тестирование - заполнение пропусков.

i = 0
times = []

# Запуск цикла для удаления и восстановления каждого параметра.
for column in testformed_data.columns:
    mis_data = testformed_data.drop(columns=[column])

    print('\n Потерян: ', column, '\n')

    for j in range(len(bns)):
        # Заполнение пропусков в сетях.
        tic = time.time()
        result_data = bns[j].predict(mis_data)
        print('\n\n', methods[j], 'Время выполнения: ', time.time()-tic)
        times.append(time.time()-tic)

        result_data.reset_index(drop=True, inplace=True)
        testformed_data.reset_index(drop=True, inplace=True)

        # Проверка типа утеряного параметра.
        if vk_data[column].dtypes == 'object':
            regime = le[i].inverse_transform(result_data.values.astype(int))
                
            # Сравнение категориальных признаков.
            print('\n Accuracy: ', accuracy_score(x_test[column], regime), '\n')

        else:
            parameters = ['age', 'mean_tr',	'median_tr', 'tr_per_month']
            if column in parameters:
                concat_data = testformed_data[parameters]
                concat_data[column] = result_data[column]

                porosity = est.inverse_transform(concat_data)
                check_data = pd.DataFrame(data=porosity, columns=parameters)
                
            else:
                check_data = result_data

        # Подсчет среднеквадратической ошибки.
        print('\n Среднеквадратическая ошибка: ', mean_squared_error(check_data[column], x_test[column], squared=False), '\n')
    
    if vk_data[column].dtypes == 'object':
        i += 1

In [ ]:
times_K2 = []
times_BDeu = []
times_BIC = []

for i in range(0,len(times)-2,3):
    times_K2.append(times[i])
    times_BDeu.append(times[i+1])
    times_BIC.append(times[i+2])

In [ ]:
print("\n Среднее время предсказания HC+K2: ", np.mean(times_K2), "\n")
print("\n Среднее время предсказания HC+BDeu: ",np.mean(times_BDeu), "\n")
print("\n Среднее время предсказания HC+BIC: ",np.mean(times_BIC), "\n")